# Benchmarking Quasi-Probabilistic Readout Correction (QPRC) of Mid-Circuit Measurements with Randomized Benchmarking (RB) Circuits

The QPRC technique was introduced in the paper Quasi-Probabilistic Readout Correction of Mid-Circuit Measurements for Adaptive Feedback via Measurement Randomized Compiling by Hashim et al. [arXiv:2303.04672](https://arxiv.org/pdf/2303.04672.pdf). 
As pointed out in the paper, the proposed technique could be useful for mitigating measuement errors in the context of QEC.
This notebook is for benchmarking QPRC on RB circuits and investigating possible extensions of the QPRC.

In [3]:
import cirq
import numpy as np

1. Construct error-corrected RB circuits from Cirq functions

In [4]:
from cirq import InsertStrategy, I, X, Y, Z

from cirq.experiments.qubit_characterizations import (
    _find_inv_matrix,
    _single_qubit_cliffords,
    _two_qubit_clifford,
    _two_qubit_clifford_matrices,
)

We'll use 2-qubit RB circuits and 3-qubit repetition code. Use a single, shallow circuit for testing and visualization purposes. 

In [5]:
num_logical = 2
physical_qubits = 3 * num_logical # repetition code
n_ancillas = 2 * num_logical
n_qubits = physical_qubits + n_ancillas
seed = 1
num_cliffords = 3
qubits = cirq.LineQubit.range(n_qubits)
cliffords = _single_qubit_cliffords()
rng = np.random.RandomState(seed)
trials = 1

Set up encoding with 3-qubit repetition code

In [6]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9 = list(qubits)
encoded_subcircuit0_tree= cirq.CNOT(q0, q1), cirq.CNOT(q0, q2)
encoded_subcircuit1_tree = cirq.CNOT(q5, q6), cirq.CNOT(q5, q7)
encoded_subcircuit = cirq.Circuit([encoded_subcircuit0_tree, encoded_subcircuit1_tree])

Construct detection subcircuit to be inserted into RB circuit

In [7]:
detect_subcircuit0_tree = [cirq.CNOT(q0, q4), cirq.CNOT(q1, q4), cirq.CNOT(q1, q3), cirq.CNOT(q2, q3), cirq.CNOT(q3, q4), cirq.measure([q3, q4])]
detect_subcircuit1_tree = [cirq.CNOT(q5, q9), cirq.CNOT(q6, q9), cirq.CNOT(q6, q8), cirq.CNOT(q7, q8), cirq.CNOT(q8, q9), cirq.measure([q8, q9])]
detect_subcircuit = cirq.Circuit([detect_subcircuit0_tree, detect_subcircuit1_tree])

Specify the circuit with random Cliffords, append the detection operations, append the inverse circuit operations, and append a second round of detection. 

In [8]:
clifford_group_size = 11520
log_qubits = [qubits[0], qubits[int((physical_qubits + n_ancillas)/num_logical)]]
cfd_matrices = _two_qubit_clifford_matrices(
            log_qubits[0],
            log_qubits[1],
            cliffords,
        )
rb_circuits = []
for _ in range(trials):
    idx_list = list(rng.choice(clifford_group_size, num_cliffords))
    rb_circuit = cirq.Circuit()
    for idx in idx_list:
        rb_circuit.append(
            _two_qubit_clifford(log_qubits[0], log_qubits[1], idx, cliffords)
        )
    inv_idx = _find_inv_matrix(
        cirq.protocols.unitary(rb_circuit), cfd_matrices
    )
    rb_circuit.append(detect_subcircuit)
    rb_circuit.append(
        _two_qubit_clifford(log_qubits[0], log_qubits[1], inv_idx, cliffords), strategy=InsertStrategy.INLINE,
    )
    rb_circuit.append(detect_subcircuit)
    rb_circuits.append(rb_circuit)

Combine the sub-circuits and print the circuit.

In [9]:
corrected_circ = cirq.Circuit([encoded_subcircuit, rb_circuit])
corrected_circ

0: ───@───@───X───────Y^0──────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5───X^0.5───Y^-0.5───@───Y^0.5────@───Y^0.5───Y───────X^0───────────────────────────────@───Y^0.5───X^0.5───@───────────────────X^-0.5───Y^0.5──────────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5────@───────────────────────
      │   │                    │            │                                              │            │                                                     │                   │                                           │            │                              │
1: ───X───┼────────────────────┼────────────┼──────────────────────────────────────────────┼────────────┼─────────────────────────────────────────────────────┼───────────────────┼───@───@───────────────────────────────────┼────────────┼──────────────────────────────┼───@───@───────────────
          │                    │            │                                              │            │                                                     │                   │   │   │                                   │            │                              │   │   │
2: ───────X────────────────────┼────────────┼──────────────────────────────────────────────┼────────────┼─────────────────────────────────────────────────────┼───────────────────┼───┼───┼───@───────────────────────────────┼────────────┼──────────────────────────────┼───┼───┼───@───────────
                               │            │                                              │            │                                                     │                   │   │   │   │                               │            │                              │   │   │   │
3: ────────────────────────────┼────────────┼──────────────────────────────────────────────┼────────────┼─────────────────────────────────────────────────────┼───────────────────┼───┼───X───X───@───M───────────────────────┼────────────┼──────────────────────────────┼───┼───X───X───@───M───
                               │            │                                              │            │                                                     │                   │   │           │   │                       │            │                              │   │           │   │
4: ────────────────────────────┼────────────┼──────────────────────────────────────────────┼────────────┼─────────────────────────────────────────────────────┼───────────────────X───X───────────X───M───────────────────────┼────────────┼──────────────────────────────X───X───────────X───M───
                               │            │                                              │            │                                                     │                                                               │            │
5: ───@───@───Y^0.5───X^-0.5───@───X^-0.5───@───X^0.5────Y^0.5────X^0.5───Y───────X────────@───X^-0.5───@───X^0.5───Y^0.5───X^0.5───Y^-0.5───X^-0.5───Y^0.5───@───Y^0.5───────────@────────────────────────────Y^-0.5───X^0───@───X^-0.5───@─────────────────────Y^-0.5───@───────────────────────
      │   │                                                                                                                                                                       │                                                                                       │
6: ───X───┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───@───@───────────────────────────────────────────────────────────────────────────────┼───@───@───────────────
          │                                                                                                                                                                       │   │   │                                                                               │   │   │
7: ───────X───────────────────────────────────────────────────────────

2. Add correlated measurement errors

3. Apply Randomized Compiling
Apply random Paulis and classical bit flip

In [83]:
def pauli_twirl(ancillas, seed=None):
    rng = np.random.RandomState(seed)
    random_paulis = rng.choice([I, X, Y, Z], size=len(ancillas))
    conditional_bitflip = [(X if r in [X, Y] else I) for r in random_paulis] # TODO must change to classical logic for HW or complex simulated noise models
    return [(p.on(ancillas[r]), conditional_bitflip[r].on(ancillas[r])) for r, p in enumerate(random_paulis)]

In [93]:
def qprc_ops(ancillas, noise_level, seed=None):
    rng = np.random.RandomState(seed)
    ops = [I, X]
    sampled_ops = rng.choice(ops, size=len(ancillas), p=[1 - noise_level, noise_level])
    return [s.on(ancillas[q]) for q, s in enumerate(sampled_ops)]

In [43]:
def detect(qubits):
    q0, q1, q2, q3, q4, q5, q6, q7, q8, q9  = qubits
    detect_subcircuit0_tree = [cirq.CNOT(q0, q4), cirq.CNOT(q1, q4), cirq.CNOT(q1, q3), cirq.CNOT(q2, q3), cirq.CNOT(q3, q4), cirq.measure([q3, q4])]
    detect_subcircuit1_tree = [cirq.CNOT(q5, q9), cirq.CNOT(q6, q9), cirq.CNOT(q6, q8), cirq.CNOT(q7, q8), cirq.CNOT(q8, q9), cirq.measure([q8, q9])]
    return cirq.Circuit([detect_subcircuit0_tree, detect_subcircuit1_tree])

In [44]:
testqbits = cirq.LineQubit.range(10)
test_detect = detect(testqbits)
test_detect

0: ───@───────────────────────
      │
1: ───┼───@───@───────────────
      │   │   │
2: ───┼───┼───┼───@───────────
      │   │   │   │
3: ───┼───┼───X───X───@───M───
      │   │           │   │
4: ───X───X───────────X───M───

5: ───@───────────────────────
      │
6: ───┼───@───@───────────────
      │   │   │
7: ───┼───┼───┼───@───────────
      │   │   │   │
8: ───┼───┼───X───X───@───M───
      │   │           │   │
9: ───X───X───────────X───M───

In [39]:
testqbits = cirq.LineQubit.range(4)
twirls = pauli_twirl(testqbits, 2, 1)
cirq.Circuit(list(zip(*twirls[:4]))[0], cirq.measure(testqbits), list(zip(*twirls[4:]))[0], cirq.measure(testqbits))

0: ───X───M───Z───M───
          │       │
1: ───Z───M───X───M───
          │       │
2: ───I───M───Z───M───
          │       │
3: ───I───M───X───M───

In [94]:
def mitigated_detect(qubits, noise_level):
    q0, q1, q2, q3, q4, q5, q6, q7, q8, q9  = qubits
    twirl = pauli_twirl([q3, q4, q8, q9], 1)
    bitflip = list(zip(*twirl))[1]
    detect_subcircuit0_tree = [cirq.CNOT(q0, q4), cirq.CNOT(q1, q4), cirq.CNOT(q1, q3), cirq.CNOT(q2, q3), cirq.CNOT(q3, q4), list(zip(*twirl[0:2]))[0], qprc_ops([q3, q4], noise_level), cirq.measure([q3, q4]), bitflip[0:2]]
    detect_subcircuit1_tree = [cirq.CNOT(q5, q9), cirq.CNOT(q6, q9), cirq.CNOT(q6, q8), cirq.CNOT(q7, q8), cirq.CNOT(q8, q9), list(zip(*twirl[2:4]))[0], qprc_ops([q8, q9], noise_level), cirq.measure([q8, q9]), bitflip[2:4]]
    return cirq.Circuit([detect_subcircuit0_tree, detect_subcircuit1_tree], strategy=InsertStrategy.EARLIEST)

In [96]:
tqbits = cirq.LineQubit.range(10)
test_detect_mitigated = mitigated_detect(tqbits, 0.2)
test_detect_mitigated

0: ───@───────────────────────────────────
      │
1: ───┼───@───@───────────────────────────
      │   │   │
2: ───┼───┼───┼───@───────────────────────
      │   │   │   │
3: ───┼───┼───X───X───@───X───I───M───X───
      │   │           │           │
4: ───X───X───────────X───Z───I───M───I───

5: ───@───────────────────────────────────
      │
6: ───┼───@───@───────────────────────────
      │   │   │
7: ───┼───┼───┼───@───────────────────────
      │   │   │   │
8: ───┼───┼───X───X───@───I───X───M───I───
      │   │           │           │
9: ───X───X───────────X───I───I───M───I───

In [ ]:
def rb_with_rc_qprc(ancillas, samples, noise_level, seed):
    

    clifford_group_size = 11520
    log_qubits = [qubits[0], qubits[int((physical_qubits + ancillas)/num_logical)]]
    cfd_matrices = _two_qubit_clifford_matrices(
            log_qubits[0],
            log_qubits[1],
            cliffords,
        )
    rc_circuits = []
    # list of Pauli sub-circuits to sample from 

    for _ in range(samples):
        idx_list = list(rng.choice(clifford_group_size, num_cliffords))
        rb_circuit = cirq.Circuit()
        for idx in idx_list:
            rb_circuit.append(
                _two_qubit_clifford(log_qubits[0], log_qubits[1], idx, cliffords)
            )
        inv_idx = _find_inv_matrix(
            cirq.protocols.unitary(rb_circuit), cfd_matrices
        )
        rb_circuit.append(detect_subcircuit)
        rb_circuit.append(
            _two_qubit_clifford(log_qubits[0], log_qubits[1], inv_idx, cliffords), strategy=InsertStrategy.INLINE,
        )
        rb_circuit.append(detect_subcircuit)
        rc_circuits.append(rb_circuit)

In [ ]:
twirled_detect0 = (cirq.Circuit(pauli(q3), pauli(q4)), bit_flip) 
twirled_detect1 = (cirq.Circuit(pauli(q8), pauli(q9)), bit_flip)

Detect subcircuit should have (independently) randomly sampled Paulis 


In [ ]:
clifford_group_size = 11520
log_qubits = [qubits[0], qubits[int((physical_qubits + ancillas)/num_logical)]]
cfd_matrices = _two_qubit_clifford_matrices(
            log_qubits[0],
            log_qubits[1],
            cliffords,
        )
rc_circuits = []
# list of Pauli sub-circuits to sample from 

for _ in range(samples):
    idx_list = list(rng.choice(clifford_group_size, num_cliffords))
    rb_circuit = cirq.Circuit()
    for idx in idx_list:
        rb_circuit.append(
            _two_qubit_clifford(log_qubits[0], log_qubits[1], idx, cliffords)
        )
    inv_idx = _find_inv_matrix(
        cirq.protocols.unitary(rb_circuit), cfd_matrices
    )
    rb_circuit.append(detect_subcircuit)
    rb_circuit.append(
        _two_qubit_clifford(log_qubits[0], log_qubits[1], inv_idx, cliffords), strategy=InsertStrategy.INLINE,
    )
    rb_circuit.append(detect_subcircuit)
    rc_circuits.append(rb_circuit)

4. Apply PEC after RC and before measurement

5. Once this workflow is working, try noise scaling at, e.g. different sampling overheads, noise levels in representations, etc.